In [2]:
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

##### ORIGINAL STEPS TO READ IN ALVARO CSVS

In [ ]:
# df15 = pd.read_csv('../clean_csv_files/alvaro_15.csv')
# df16 = pd.read_csv('../clean_csv_files/alvaro_16.csv')
# df17 = pd.read_csv('../clean_csv_files/alvaro_17.csv')
# df18 = pd.read_csv('../clean_csv_files/alvaro_18.csv')
# df19 = pd.read_csv('../clean_csv_files/alvaro_19.csv')

In [ ]:
# df_combined_1 = pd.concat([df15, df16, df17, df18, df19])
# df_combined_1.info()

In [ ]:
# df_combined_1.to_csv('../combined_csvs/alvaro_combined.csv', index=False)

##### READING IN COMBINED CSVS

In [4]:
combined_df = pd.read_csv('../combined_csvs/combined_vg_sales_summary.csv')

In [5]:
# New dataframe containing data for preprocessing
df_new = combined_df[['Genre','Summary']].copy()
df_new

,Genre,Summary
0,Sports,Wii Sports was developed by Nintendo to show t...
1,Platform,"For more than 25 years, the Super Mario series..."
2,Racing,Mario Kart Wii includes 16 new courses and 16 ...
3,Shooter,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...
4,Sports,"Wii Sports Resort is an immersive, expansive a..."
...,...,...
18854,Simulation,
18855,Shooter,"Infiltrate terrorists' positions, acquire crit..."
18856,Fighting,
18857,Adventure,


In [6]:
df_new['Genre'].value_counts()

Action              2786
Sports              2394
Misc                2123
Adventure           1789
Role-Playing        1728
Shooter             1475
Racing              1359
Simulation          1065
Platform            1061
Fighting             897
Strategy             831
Puzzle               706
Action-Adventure     316
Music                131
Visual Novel         110
MMO                   43
Party                 36
Sandbox                4
Education              3
Board Game             2
Name: Genre, dtype: int64

In [7]:
# combine genres with low counts into misc
# combine repeating genres, Action and Action
df_new.replace({'Genre': 
             {'Board Game':'Misc', 
              'Party':'Misc',
              'MMO': 'Misc',
              'Sandbox':'Misc',
              'Education':'Misc'
#               'Music':'Misc'
#               'Adventure':'Action-Adventure', 
#               'Action':'Action-Adventure'
             }}, 
            inplace= True)
df_new['Genre'].value_counts()

Action              2786
Sports              2394
Misc                2211
Adventure           1789
Role-Playing        1728
Shooter             1475
Racing              1359
Simulation          1065
Platform            1061
Fighting             897
Strategy             831
Puzzle               706
Action-Adventure     316
Music                131
Visual Novel         110
Name: Genre, dtype: int64

In [8]:
# encode genres as a value in a new column
df_new['Genre_ID'] = df_new['Genre'].factorize()[0]
genre_df = df_new[['Genre', 'Genre_ID']].drop_duplicates()
# genre_df
df_new.head()

,Genre,Summary,Genre_ID
0,Sports,Wii Sports was developed by Nintendo to show t...,0
1,Platform,"For more than 25 years, the Super Mario series...",1
2,Racing,Mario Kart Wii includes 16 new courses and 16 ...,2
3,Shooter,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,3
4,Sports,"Wii Sports Resort is an immersive, expansive a...",0


In [9]:
# fig = plt.figure(figsize=(8,6))
# df_new.groupby('Genre').Summary.count().sort_values().plot.barh(
#     ylim=0, color='cornflowerblue', title= 'Genres for Each Summary\n')
# plt.xlabel('Summaries per Genre', fontsize = 10);

In [10]:
tfidf = TfidfVectorizer(sublinear_tf=True,
                        ngram_range=(1, 4), 
                        stop_words='english')

# We transform each complaint into a vector
features = tfidf.fit_transform(df_new['Summary']).toarray()
labels = df_new['Genre_ID']


MemoryError: Unable to allocate 279. GiB for an array with shape (18859, 1988563) and data type float64

In [ ]:
features.shape

In [ ]:
genre_to_id = dict(genre_df.values)
genre_to_id

In [ ]:
id_to_genre = dict(genre_df[['Genre_ID', 'Genre']].values)
id_to_genre

In [ ]:
# extracting the most common single and double word-phrases (biigram) in each genre
N = 10
# wordcloud = []

for genre, genre_id in sorted(genre_to_id.items()):
    
    
    unigram = {}
#     bigram = {}
#     trigram = {}
    
    features_chi2 = chi2(features, labels == genre_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#     data[genre] = b=unigrams
#     bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#     data[genre] = bigrams
#     trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
#     data[genre] = trigrams
        
    print("\nGenre: %s" %(genre))
    print("  * Unigram keywords are: %s" %(', '.join(unigrams[-N:])))
#     print("  * Bigram keywords are: %s" %(', '.join(bigrams[-N:])))
#     print("  * Trigram keywords are: %s" %(', '.join(trigrams[-N:])))
#     wordcloud.append(data)

In [ ]:
print(data)

In [ ]:
wordcloud